In [ ]:
import random
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image, ImageDraw, ImageFont, ImageOps
from tqdm import tqdm

In [ ]:
"""
학습 시키기 전 bounding box가 제대로 형성이 되었는지 확인하는 부분
"""
dir_main = "/home/tlrkrwlsmd/yolov5/dataset"
filenames_image = glob(f"{dir_main}/images/train/*.jpg")[100:]
filenames_label = [filename.replace('images', 'labels').replace('jpg', 'txt') for filename in filenames_image]
classes = ["name", "number", "information"]
color = []
# color 생성하는 부분
for _ in range(3):
    c = list(np.random.choice(range(256), size=3)) + [255]
    c = tuple(c)
    color.append(c)

# annotation 정보가 제대로 저장됐는지 확인하기 위한 bounding box를 시각화하는 함수
def draw_bbox(draw, bbox, label, color=(0, 255, 0, 255), confs=None, size=15):
    font = ImageFont.load_default()
    draw.rectangle(bbox, outline=color, width =3)
    def set_alpha(color, value):
        background = list(color)
        background[3] = value
        return tuple(background)
    background = set_alpha(color, 50)
    draw.rectangle(bbox, outline=color, fill=background, width =3)
    background = set_alpha(color, 150)
    text = f"{label}" + ("" if confs==None else f":{conf:0.4}")
    text_bbox = bbox[0], bbox[1], bbox[0]+len(text)*10, bbox[1]+25
    draw.rectangle(text_bbox, outline=color, fill=background, width =3)
    draw.text((bbox[0]+5, bbox[1]+5), text, (0,0,0), font=font)

cnt = 3
for filename_image, filename_label in tqdm(zip(filenames_image, filenames_label)):
    img = Image.open(filename_image)
    img = img.resize((640, 640))
    width, height = img.size
    draw = ImageDraw.Draw(img, 'RGBA')
    with open(filename_label, 'r') as f:
        labels = f.readlines()
        # strip: 개행 문자 제거.
        labels = list(map(lambda s: s.strip().split(), labels))
    # bounding box를 그리는 부분
    for label in labels:
        # class는 0부터 시작!
        cls = int(label[0])
        print(cls)
        x, y, w, h = map(float, label[1:])
        x1, x2 = width * (x-w/2), width * (x+w/2)
        y1, y2 = height * (y-h/2), height * (y+h/2)
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        draw_bbox(draw, bbox=(x1, y1, x2, y2), label=classes[cls], color=color[cls], size=15)
    img.show()
    
    cnt -= 1
    if cnt ==0:
        break

In [ ]:
"""
학습 시키기 전 bounding box가 제대로 형성이 되었는지 확인하는 부분
"""
dir_main = "/home/tlrkrwlsmd/adcard_data/"
filenames_image = glob(f"{dir_main}/*.png")[10:]
# filenames_image = [os.path.join(dir_main, 'augment_10.png')]
filenames_label = [filename.replace('images', 'labels').replace('png', 'txt') for filename in filenames_image]
classes = ["name", "number", "information"]
color = []
# color 생성하는 부분
for _ in range(3):
    c = list(np.random.choice(range(256), size=3)) + [255]
    c = tuple(c)
    color.append(c)

print(filenames_image[:3])
# annotation 정보가 제대로 저장됐는지 확인하기 위한 bounding box를 시각화하는 함수
def draw_bbox(draw, bbox, label, color=(0, 255, 0, 255), confs=None, size=15):
    font = ImageFont.load_default()
    draw.rectangle(bbox, outline=color, width =3)
    def set_alpha(color, value):
        background = list(color)
        background[3] = value
        return tuple(background)
    # background = set_alpha(color, 200)
    # draw.rectangle(bbox, outline=color, fill=background, width =3)
    background = set_alpha(color, 150)
    text = f"{label}" + ("" if confs==None else f":{conf:0.4}")
    text_bbox = bbox[0], bbox[1], bbox[0]+len(text)*10, bbox[1]+25
    draw.rectangle(text_bbox, outline=color, fill=background, width =3)
    draw.text((bbox[0]+5, bbox[1]+5), text, (0,0,0), font=font)

cnt = 1
for filename_image, filename_label in tqdm(zip(filenames_image, filenames_label)):
    img = Image.open(filename_image)
    img = img.resize((640, 640))
    width, height = img.size
    draw = ImageDraw.Draw(img, 'RGBA')
    with open(filename_label, 'r') as f:
        labels = f.readlines()
        # strip: 개행 문자 제거.
        labels = list(map(lambda s: s.strip().split(), labels))
    # bounding box를 그리는 부분
    for label in labels:
        # class는 0부터 시작!
        cls = int(label[0])
        x, y, w, h = map(float, label[1:])
        x1, x2 = width * (x-w/2), width * (x+w/2)
        y1, y2 = height * (y-h/2), height * (y+h/2)
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        draw_bbox(draw, bbox=(x1, y1, x2, y2), label=classes[cls], color=color[cls], size=15)
    img.save("/home/tlrkrwlsmd/output2.png")
    img.show()
    
    cnt -= 1
    if cnt ==0:
        break

In [ ]:
import yaml
data = {'train': "/home/tlrkrwlsmd/yolov5/dataset/images/train",
        "val": "/home/tlrkrwlsmd/yolov5/dataset/images/valid",
        "nc": 3,
        "names": ['name', 'number', 'information']}

with open("cfg.yaml", "w") as f:
    yaml.dump(data, f)

In [ ]:
!python train.py --weights /home/tlrkrwlsmd/yolov5/yolov5s.pt --data /home/tlrkrwlsmd/yolov5/cfg.yaml --hyp /home/tlrkrwlsmd/yolov5/data/hyps/hyp.scratch-low.yaml --device 0 --cos-lr --patience 10 --batch-size 16 --label-smoothing 0.1 --workers 0

In [ ]:
!python val.py --weights /home/tlrkrwlsmd/yolov5/runs/train/exp2/weights/best.pt --data /home/tlrkrwlsmd/yolov5/cfg.yaml --device 0